In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cProfile

import numpy as np

import kraft

In [ ]:
def plot_x_y(xs, ys, xaxis_title_text, yaxis_title_text, title_text=None):

    kraft.plot_plotly(
        {
            "layout": {
                "title": {"text": title_text},
                "xaxis": {"title": {"text": xaxis_title_text}},
                "yaxis": {"title": {"text": yaxis_title_text}},
            },
            "data": [{"x": xs, "y": ys}],
        }
    )

## get_entropy

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector = np.random.random_sample(size=n)

    ys.append(kraft.get_entropy(vector))

plot_x_y(xs, ys, "N", "Entropy")

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector = np.full(n, 1)

    ys.append(kraft.get_entropy(vector))

plot_x_y(xs, ys, "N", "Entropy")

## get_kld

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector_0 = np.random.random_sample(size=n)

    vector_1 = np.random.random_sample(size=n)

    klds = kraft.get_kld(vector_0, vector_1)

    ys.append(klds.sum())

plot_x_y(xs, ys, "N", "KLD")

## get_jsd

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector_0 = np.random.random_sample(size=n)

    vector_1 = np.random.random_sample(size=n)

    jsds = kraft.get_jsd(vector_0, vector_1)[-1]

    ys.append(jsds.sum())

plot_x_y(xs, ys, "N", "JSD")

## get_zd

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector_0 = np.random.random_sample(size=n)

    vector_1 = np.random.random_sample(size=n)

    zds = kraft.get_zd(vector_0, vector_1)[-1]

    ys.append(zds.sum())

plot_x_y(xs, ys, "N", "ZD")

## get_ic

In [ ]:
for vector_0, vector_1 in (
    ((0,) * 8,) * 2,
    ((1,) * 8,) * 2,
    (np.arange(8), np.arange(8)[::-1]),
    ((0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0),) * 2,
    ((8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8),) * 2,
    ((0, 1) * 8, ((0, 1) * 8)),
    ((0, 1) * 8, ((1, 0) * 8)),
):

    vector_0 = np.asarray(vector_0)

    vector_1 = np.asarray(vector_1)

    print(vector_0)

    print(vector_1)

    print(kraft.get_ic(vector_0, vector_1))

In [ ]:
n = 2

vector_0 = np.asarray((0, 0, 1, 1, 2, 2))

print(vector_0)

for factor in (1, 2, 1 / 2, 10, 1 / 10):

    vector_1 = vector_0 * factor

    print(vector_1)

    print(kraft.get_ic(vector_0, vector_1))

In [ ]:
xs = []

ys = []

for power in range(10):

    n = 2 ** power

    xs.append(n)

    vector_0 = np.random.random_sample(size=n)

    vector_1 = np.random.random_sample(size=n)

    ys.append(kraft.get_ic(vector_0, vector_1))

plot_x_y(xs, ys, "N", "IC")

In [ ]:
n = 100

vector_0 = np.random.random_sample(size=n)

vector_1 = np.random.random_sample(size=n)

print(kraft.get_ic(vector_0, vector_1))

In [ ]:
%%timeit

kraft.get_ic(vector_0, vector_1)

In [ ]:
cProfile.run("kraft.get_ic(vector_0, vector_1)", sort="cumtime")